In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dat = pd.read_parquet("/content/drive/MyDrive/SCCM_Datathon_Team1/data.parquet")

In [17]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360029 entries, 0 to 360028
Data columns (total 50 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   patientunitstayid          360029 non-null  Int64  
 1   core_glucose_offset        360029 non-null  Int64  
 2   core_glucose               360029 non-null  float64
 3   poc_glucose_offset         360029 non-null  Int64  
 4   poc_glucose                360029 non-null  float64
 5   delta_offset               360029 non-null  Int64  
 6   delta_glucose              360029 non-null  float64
 7   uniquepid                  360029 non-null  object 
 8   patienthealthsystemstayid  360029 non-null  Int64  
 9   unitvisitnumber            360029 non-null  Int64  
 10  hospitalid                 360029 non-null  Int64  
 11  region                     334291 non-null  object 
 12  unittype                   360029 non-null  object 
 13  hospitaladmitoffset        36

In [18]:
dat

,patientunitstayid,core_glucose_offset,core_glucose,poc_glucose_offset,poc_glucose,delta_offset,delta_glucose,uniquepid,patienthealthsystemstayid,unitvisitnumber,...,bun,delta_bun,creatinine,delta_creatinine,lactate,delta_lactate,insulin_past24h,steroids_past24h,vasopressors_past24h,mech_vent_past12h
0,656658,-300,45.0,-334,34.0,34,11.0,006-179266,524521,1,...,36.0,0,5.02,0,NaN,<NA>,0,0,0,0
1,724732,9123,43.0,9117,36.0,6,7.0,006-144318,565216,2,...,27.0,0,5.17,0,NaN,<NA>,1,0,0,1
2,1786193,2486,48.0,2528,40.0,-42,8.0,018-27002,1402676,1,...,18.0,0,1.16,0,NaN,<NA>,0,0,0,0
3,1808587,773,170.0,757,40.0,16,130.0,018-95296,1422309,1,...,65.0,0,12.81,0,NaN,<NA>,1,0,1,1
4,1579198,371,41.0,412,44.0,-41,-3.0,016-2799,1220596,1,...,NaN,<NA>,3.29,0,NaN,<NA>,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360024,492705,4430,154.0,4426,167.0,4,-13.0,005-13213,417299,1,...,15.0,0,0.50,0,NaN,<NA>,0,0,0,0
360025,481443,76,196.0,66,167.0,10,29.0,005-15450,408073,1,...,NaN,<NA>,NaN,<NA>,3.6,175,0,0,0,0
360026,1026147,-5720,186.0,-5738,167.0,18,19.0,008-26178,759014,1,...,12.0,0,1.00,0,NaN,<NA>,0,0,0,0
360027,3085392,8945,147.0,8932,167.0,13,-20.0,030-326,2505337,1,...,39.0,0,NaN,<NA>,NaN,<NA>,1,0,1,1


In [6]:
hospital_counts = dat[['hospitalid', 'uniquepid']].groupby('hospitalid').count().reset_index()
keep_centers = hospital_counts[hospital_counts['uniquepid'] > 1000]['hospitalid']

In [7]:
df = dat[dat.hospitalid.isin(keep_centers)].copy()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325402 entries, 0 to 360028
Data columns (total 50 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   patientunitstayid          325402 non-null  Int64  
 1   core_glucose_offset        325402 non-null  Int64  
 2   core_glucose               325402 non-null  float64
 3   poc_glucose_offset         325402 non-null  Int64  
 4   poc_glucose                325402 non-null  float64
 5   delta_offset               325402 non-null  Int64  
 6   delta_glucose              325402 non-null  float64
 7   uniquepid                  325402 non-null  object 
 8   patienthealthsystemstayid  325402 non-null  Int64  
 9   unitvisitnumber            325402 non-null  Int64  
 10  hospitalid                 325402 non-null  Int64  
 11  region                     302944 non-null  object 
 12  unittype                   325402 non-null  object 
 13  hospitaladmitoffset        32

In [9]:
# ## These cols are NA if zero, so zero fill them
# for col in ['vasopressors_past24h', 'steroids_past24h', 'insulin_past24h']:
#   df[col] = df[col].fillna(0)

for col in ['age', 'gender', 'ethnicity']:
  df.loc[df[col] == "", col] = np.nan

# Make an indicator var for age > 89 and turn age into numeric
df['age_gt_89'] = df['age'] == "> 89"
df.loc[(df['age'] == "> 89"), "age"] = "90"
df['age'] = pd.to_numeric(df['age'])

df.loc[df['admissionheight'] < 40, 'admissionheight'] = np.nan
df.loc[df['admissionheight'] > 240, 'admissionheight'] = np.nan

In [10]:
df.loc[df['ethnicity'].isna(), 'ethnicity'] = "Other/Unknown"

In [11]:
CAT_COLS = ['ethnicity', 'region', 'unittype']
for col in CAT_COLS:
  df[col] = df[col].astype('category')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325402 entries, 0 to 360028
Data columns (total 51 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   patientunitstayid          325402 non-null  Int64   
 1   core_glucose_offset        325402 non-null  Int64   
 2   core_glucose               325402 non-null  float64 
 3   poc_glucose_offset         325402 non-null  Int64   
 4   poc_glucose                325402 non-null  float64 
 5   delta_offset               325402 non-null  Int64   
 6   delta_glucose              325402 non-null  float64 
 7   uniquepid                  325402 non-null  object  
 8   patienthealthsystemstayid  325402 non-null  Int64   
 9   unitvisitnumber            325402 non-null  Int64   
 10  hospitalid                 325402 non-null  Int64   
 11  region                     302944 non-null  category
 12  unittype                   325402 non-null  category
 13  hospitaladmito

In [13]:
df.to_parquet("/content/drive/MyDrive/SCCM_Datathon_Team1/data_clean.parquet")

In [14]:
df

,patientunitstayid,core_glucose_offset,core_glucose,poc_glucose_offset,poc_glucose,delta_offset,delta_glucose,uniquepid,patienthealthsystemstayid,unitvisitnumber,...,delta_bun,creatinine,delta_creatinine,lactate,delta_lactate,insulin_past24h,steroids_past24h,vasopressors_past24h,mech_vent_past12h,age_gt_89
0,656658,-300,45.0,-334,34.0,34,11.0,006-179266,524521,1,...,0,5.02,0,NaN,<NA>,0,0,0,0,False
1,724732,9123,43.0,9117,36.0,6,7.0,006-144318,565216,2,...,0,5.17,0,NaN,<NA>,1,0,0,1,False
3,1808587,773,170.0,757,40.0,16,130.0,018-95296,1422309,1,...,0,12.81,0,NaN,<NA>,1,0,1,1,False
4,1579198,371,41.0,412,44.0,-41,-3.0,016-2799,1220596,1,...,<NA>,3.29,0,NaN,<NA>,0,0,1,0,False
6,2469979,4727,96.0,4762,54.0,-35,42.0,022-124053,1962429,2,...,0,1.30,0,NaN,<NA>,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360022,3163238,1450,208.0,1454,167.0,-4,41.0,032-30324,2572279,2,...,0,1.41,0,NaN,<NA>,1,0,0,0,False
360025,481443,76,196.0,66,167.0,10,29.0,005-15450,408073,1,...,<NA>,NaN,<NA>,3.6,175,0,0,0,0,False
360026,1026147,-5720,186.0,-5738,167.0,18,19.0,008-26178,759014,1,...,0,1.00,0,NaN,<NA>,0,0,0,0,False
360027,3085392,8945,147.0,8932,167.0,13,-20.0,030-326,2505337,1,...,0,NaN,<NA>,NaN,<NA>,1,0,1,1,False
